# Recipe Rating Machine Learning

In [ ]:
import warnings
warnings.simplefilter('ignore')

# %matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import sqlite3
from sqlite3 import Error

# Dataset:  epicurious_ratings.csv

Source: Kaggle (https://www.kaggle.com/hugodarwood/epirecipes#full_format_recipes.json)

Recipe information lifted from: http://www.epicurious.com/recipes-menus

Description: Recipe data information, containing categorical and nutritional information

Variables/Columns  
TITLE  
RATING: 0.000 to 5.000  
  (0.0 - 0.999 = Eww! / 1.0 - 1.999 = Better than not eating / 2.0 - 2.999 = Edible / I guess, 3.000 - 3.999 = Pretty darn good / 4.000 - 4.999 = Git in ma belleh)
CALORIES   
FAT  
SODIUM


## Potential future, interesting work to add in...  
=> Distribution of ratings..  
  => How many ingredients per recipe and does it correlate to an increase in star ratings  
=> Locate columns with counts of "1"s greater than 1000 (to identify specialty diets and ingrediants) ? Do a for loop and summation  


## Questions...


optiomal 
Normal linear regression
Use each variable in a neural network... 
Keras 

In [ ]:
# Read the csv file into a pandas DataFrame
ratings = pd.read_csv('Resources/epicurious_ratings.csv')
ratings.head(2)

In [ ]:
ratings.shape

In [ ]:
# Create new dataframe with NaN values removed
ratings = ratings.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
ratings

In [ ]:
# delete all rows with column 'calories' has value > 2000
outliersCalHigh = ratings[ (ratings['calories'] >= 2000.00) ].index
ratings.drop(outliersCalHigh , inplace=True)
ratings

In [ ]:
# delete all rows with column 'calories' has value 0 to 100
outliersCalLow = ratings[ (ratings['calories'] >= 0) & (ratings['calories'] <= 100) ].index
ratings.drop(outliersCalLow , inplace=True)
ratings.head(2)

In [ ]:
# delete all rows with column 'calories' has value 0 to 100
outliersSodium = ratings[ (ratings['sodium'] >= 10000) ].index
ratings.drop(outliersSodium , inplace=True)
ratings.head(2)

In [ ]:
ratings.shape

In [ ]:
# Reset the index on the new dataframe
ratings.reset_index(inplace=True)
ratings

In [ ]:
ratings["sodiumG"] = ratings["sodium"]*.001

In [ ]:
ratings

In [ ]:
keep_cols = ["calories", "protein", "fat", "sodiumG", "rating"]
# Potential side chart / analysis - speciality diet ratings / specific ingrediants include: "vegan", "vegetarian", "dairy free", "wheat/gluten-free", 
# "peanut free", "meat", "alcoholic", "chocolate", "cheese"

In [ ]:
ratings = ratings[keep_cols]
ratings.head(2)

In [ ]:
# write this back to csv and for upload to gitHub
ratings.to_csv(r"C:\Users\Deanna\Desktop\Project3\Recipe_Rating\Resources\epi_ratings_columns_removed.csv")  

In [ ]:
ratings.shape

In [ ]:
target = ratings["rating"]
target.shape

In [ ]:
data = ratings.drop("rating", axis=1)
feature_names = data.columns
data.head()

In [ ]:
rating_group=ratings.groupby("rating")
print(rating_group["calories"].count())

# LabelEncode FULL dataset ratings

In [ ]:
data = ratings.values
X = data[:, 0:4]
y = data[:, 4]

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y)
encoded_y = label_encoder.transform(y)

In [ ]:
for label, original_class in zip(encoded_y, y):
    print('Original Class: ' + str(original_class))
    print('Encoded Label: ' + str(label))
    print('-' * 12)

In [ ]:
encoded_y

In [ ]:
y

# One-hot Encode Ratings -- FULL DATA

In [ ]:
from keras.utils import to_categorical

# Step 2: One-hot encoding
one_hot_y = to_categorical(encoded_y)
one_hot_y

In [ ]:
one_hot_y[0]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, one_hot_y, random_state=42)
y_test

In [ ]:
X_train.shape

# Use StandardScalar to scale FULL DATA set

In [ ]:
from sklearn.preprocessing import StandardScaler

# Create a StandardScater model and fit it to the training data
X_scaler = StandardScaler().fit(X_train)

In [ ]:
# Transform the training and testing data using the X_scaler

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Run RandomForest Classifier on FULL dataset, Calories and Sodium seem to be greatest prediction

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=2000)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

In [ ]:
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

# Create Neural Network with FULL data set, no meaningful data

In [ ]:
# first, create a normal neural network with 4 inputs, 12 hidden nodes, and 8 outputs
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=12, activation='relu', input_dim=4))
model.add(Dense(units=8, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Fit the model to the training data
model.fit(
    X_train_scaled,
    y_train,
    epochs=100,
    shuffle=True,
    verbose=2
)

In [ ]:
# Evaluate the model using the testing data
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# first, create a normal neural network with 4 inputs, 12 hidden nodes, and 8 outputs
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

deep_model = Sequential()
deep_model.add(Dense(units=12, activation='relu', input_dim=4))
deep_model.add(Dense(units=12, activation='relu'))
deep_model.add(Dense(units=8, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Fit the model to the training data
model.fit(
    X_train_scaled,
    y_train,
    epochs=100,
    shuffle=True,
    verbose=2
)

In [ ]:
# Evaluate the model using the testing data
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Try KNN - FULL Data Set, no meaningful grouping

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# Loop through different k values to see which has the highest accuracy
# Note: We only use odd numbers because we don't want any ties
train_scores = []
test_scores = []
for k in range(1, 20, 2):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    train_score = knn.score(X_train, y_train)
    test_score = knn.score(X_test, y_test)
    train_scores.append(train_score)
    test_scores.append(test_score)
    print(f"k: {k}, Train/Test Score: {train_score:.3f}/{test_score:.3f}")
    
    
plt.plot(range(1, 20, 2), train_scores, marker='o')
plt.plot(range(1, 20, 2), test_scores, marker="x")
plt.xlabel("k neighbors")
plt.ylabel("Testing accuracy Score")
plt.show()

In [ ]:
# Note that k: 13 seems to be the best choice for this dataset
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
print('k=13 Test Acc: %.3f' % knn.score(X_test, y_test))

# DESSERT DATA ONLY - Select 5 key columns of desserts

In [ ]:
desserts = ratings.loc[ratings["dessert"] == 1, :]
desserts.head()

In [ ]:
# Reset the index on the new dataframe
desserts.reset_index(inplace=True)
desserts.head(2)

In [ ]:
desserts["fatCalRatio"] = desserts["fat"]/desserts["calories"]
desserts["sodiumCalRatio"] = desserts["sodium"]/desserts["calories"]
desserts["fatSodiumRatio"] = desserts["fat"]/desserts["sodium"]


In [ ]:
keep_cols_fat_ratio = ["calories", "fat", "sodium", "fatCalRatio", "rating"]
keep_cols_sodium_ratio = ["calories", "fat", "sodium", "sodiumCalRatio", "rating"]
keep_cols_fat_sodium = ["calories", "protein", "fat", "sodium", "fatSodiumRatio", "rating"]
keep_cols_basic = ["calories", "fat", "sodium", "rating"]
keep_cols_ratios_only = ["fatCalRatio", "sodiumCalRatio", "rating"]

In [ ]:
dessertsDataWithRatio = desserts[keep_cols_ratios_only]
dessertsData = desserts[keep_cols_basic]
dessertsDataWithRatio

In [ ]:
dessertsDataWithRatio["fatSodiumRatio"] = dessertsDataWithRatio["fatSodiumRatio"].astype(float64).map("{:.4f}".format)


In [ ]:
dessertsDataWithRatio.head(40)

# LabelEncode ratings for desserts

In [ ]:
dataWithRatio = dessertsDataWithRatio.values
X = dataWithRatio[:, 0:2]
y = dataWithRatio[:, 2]
X

In [ ]:
dataD = dessertsData.values
X = dataD[:, 0:3]
y = dataD[:, 3]
X

In [ ]:
X.dtype

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y)
encoded_y = label_encoder.transform(y)
encoded_y.shape

In [ ]:
for label, original_class in zip(encoded_y, y):
    print('Original Class: ' + str(original_class))
    print('Encoded Label: ' + str(label))
    print('-' * 12)

In [ ]:
encoded_y
# y

# One-hot encode ratings for DESSERTS only

In [ ]:
from keras.utils import to_categorical

# Step 2: One-hot encoding
one_hot_y = to_categorical(encoded_y)
one_hot_y

In [ ]:
one_hot_y[0]

# Train, Test, Split DESSERT data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, one_hot_y, random_state=42)
X_train

# StandardScaler and fit, transform DESSERT training data

In [ ]:
from sklearn.preprocessing import StandardScaler

# Create a StandardScaler model and fit it to the training data
X_scaler = StandardScaler().fit(X_train)

In [ ]:
# Transform the training and testing data using the X_scaler

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# RandomForest DESSERT data

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=2000)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

In [ ]:
# feature_names = dessertsData.columns
feature_names = dessertsDataWithRatio.columns

In [ ]:
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

In [ ]:
# first, create a normal neural network with 4 inputs, 12 hidden nodes, and 8 outputs
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=6, activation='relu', input_dim=2))
model.add(Dense(units=8, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Fit the model to the training data
model.fit(
    X_train_scaled,
    y_train,
    epochs=100,
    shuffle=True,
    verbose=2
)

In [ ]:
# Evaluate the model using the testing data
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# first, create a normal neural network with 4 inputs, 12 hidden nodes, and 8 outputs
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

deep_model = Sequential()
deep_model.add(Dense(units=9, activation='relu', input_dim=3))
deep_model.add(Dense(units=9, activation='relu'))
deep_model.add(Dense(units=8, activation='softmax'))

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Fit the model to the training data
model.fit(
    X_train_scaled,
    y_train,
    epochs=100,
    shuffle=True,
    verbose=2
)

In [ ]:
# Evaluate the model using the testing data
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# Loop through different k values to see which has the highest accuracy
# Note: We only use odd numbers because we don't want any ties
train_scores = []
test_scores = []
for k in range(1, 20, 2):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    train_score = knn.score(X_train, y_train)
    test_score = knn.score(X_test, y_test)
    train_scores.append(train_score)
    test_scores.append(test_score)
    print(f"k: {k}, Train/Test Score: {train_score:.3f}/{test_score:.3f}")
    
    
plt.plot(range(1, 20, 2), train_scores, marker='o')
plt.plot(range(1, 20, 2), test_scores, marker="x")
plt.xlabel("k neighbors")
plt.ylabel("Testing accuracy Score")
plt.show()